In [172]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StructField, BooleanType, LongType, IntegerType

from pyspark.sql.types import *
import time
import json


spark = (SparkSession
.builder
.master('local')
.appName('wiki-changes-event-consumer')
# Add kafka package
.config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,net.snowflake:spark-snowflake_2.12:2.10.0-spark_3.2")
.getOrCreate())
sc = spark.sparkContext



sfOptions = {
 "sfURL" : "zu02863.ap-south-1.aws.snowflakecomputing.com",
 "sfAccount" : "zu02863.ap-south-1.aws",
 "sfUser" : "coolkeonjhar",
 "sfPassword" : "Amrita@123",
 "sfDatabase" : "NIMBUS_MAPS",
 "sfSchema" : "RAW_LAYER",
 "sfWarehouse" : "compute_wh",
 "sfRole" : "ACCOUNTADMIN"
}
 
    
df = spark.read.format('snowflake') \
  .options(**sfOptions) \
  .option("query",  "select UID,ADDRESS,CITY,ZIPCODE,DECISION,CURRENT_TIMESTAMP() AS CREATE_TS from NIMBUS_MAPS.STAGING_DATA_LAKE.DECISION_DATA_STG where create_ts >(select nvl(max(CREATE_TS),to_timestamp('2001-01-01 00:00:00','YYYY-MM-DD HH24:MI:SS')) FROM NIMBUS_MAPS.RAW_LAYER.DECISION_DATA_RAW)") \
  .load()

df.printSchema()

root
 |-- UID: decimal(38,0) (nullable = false)
 |-- ADDRESS: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- ZIPCODE: string (nullable = true)
 |-- DECISION: string (nullable = true)
 |-- CREATE_TS: timestamp (nullable = false)



In [173]:
type(df)

pyspark.sql.dataframe.DataFrame

In [69]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

In [75]:
duplicates_count=df_dupli.count()
if duplicates_count>0:
    print(f'it has {duplicates_count} duplicates')

it has 1 duplicates


In [71]:
df.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions).option("dbtable", "NIMBUS_MAPS.RAW_LAYER.DECISION_DATA_RAW").mode("append").options(header=True).save()

In [76]:
df_dupli.show()

+-------+--------------------+---------+-------+--------+--------------------+-----+
|    UID|             ADDRESS|     CITY|ZIPCODE|DECISION|           CREATE_TS|count|
+-------+--------------------+---------+-------+--------+--------------------+-----+
|8902048|3808 Joseph Forge...|Ceciliano|  33562|       C|+0530 2022-02-27 ...|    2|
+-------+--------------------+---------+-------+--------+--------------------+-----+



In [79]:
 df_dupli.select('UID','CREATE_TS').write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions).option("dbtable", "NIMBUS_MAPS.RAW_LAYER.log_duplicates").mode("append").options(header=True).save()

DataFrame[UID: decimal(38,0)]

In [155]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext
import pyspark.sql.functions as F


import time
import json
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

spark = (SparkSession
.builder
.master('local')
.appName('wiki-changes-event-consumer')
# Add kafka package
.config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,net.snowflake:spark-snowflake_2.12:2.10.0-spark_3.2")
.getOrCreate())
sc = spark.sparkContext



sfOptions = {
 "sfURL" : "zu02863.ap-south-1.aws.snowflakecomputing.com",
 "sfAccount" : "zu02863.ap-south-1.aws",
 "sfUser" : "coolkeonjhar",
 "sfPassword" : "Amrita@123",
 "sfDatabase" : "NIMBUS_MAPS",
 "sfSchema" : "RAW_LAYER",
 "sfWarehouse" : "compute_wh",
 "sfRole" : "ACCOUNTADMIN"
}
 
    
df = spark.read.format('snowflake') \
  .options(**sfOptions) \
  .option("query",  "select UID,ADDRESS,CITY,ZIPCODE,DECISION,CURRENT_TIMESTAMP() AS CREATE_TS from NIMBUS_MAPS.RAW_LAYER.DECISION_DATA_RAW where create_ts >(select nvl(max(CREATE_TS),to_timestamp('2001-01-01 00:00:00','YYYY-MM-DD HH24:MI:SS')) FROM nimbus_maps.presentation.decision_data_pres)") \
  .load()

print('printing count')
print(df.count())

if df.count()==0:
    print('No data to process')
    sc.stop()
    quit()
    

    


printing count
4555


In [156]:
def isGoodDecision(decision):
    verdict = ''
    if decision.upper()=='A':
        verdict=True
    else:
        verdict=False
    return verdict

In [169]:
df4.select('create_ts').show()

+--------------------+
|           create_ts|
+--------------------+
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
|+0530 2022-02-27 ...|
+--------------------+
only showing top 20 rows



In [178]:
udfsomefunc = F.lower(F.udf(isGoodDecision, StringType()))
df3 = df.withColumn("isGoodDecision", udfsomefunc("DECISION"))
df4 = df3.select('UID','DECISION','isGoodDecision','create_ts')

TypeError: Invalid argument, not a string or column: <function isGoodDecision at 0x7f13261791f0> of type <class 'function'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [177]:
df4



DataFrame[UID: decimal(38,0), DECISION: string, isGoodDecision: string, create_ts: timestamp]

In [181]:
Logger= spark._jvm.org.apache.log4j.Logger
mylogger = Logger.getLogger(__name__)



In [182]:
mylogger.info("some info trace")